In [ ]:
%pip install mlflow pymongo tqdm

Python interpreter will be restarted.
  Using cached mlflow-2.4.1-py3-none-any.whl (18.1 MB)
  Using cached pymongo-4.3.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (492 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached docker-6.1.3-py3-none-any.whl (148 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached alembic-1.11.1-py3-none-any.whl (224 kB)
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached importlib_metadata-6.6.0-py3-none-any.whl (22 kB)
  Using cached SQLAlchemy-2.0.16-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.7 MB)
  Usin

In [ ]:
from pymongo import MongoClient
MONGO_CONN = 'mongodb+srv://<username>:<password>@retail-demo.2wqno.mongodb.net/?retryWrites=true&w=majority'
client = MongoClient(MONGO_CONN)

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window, WindowSpec
from pyspark.sql.functions import struct

import mlflow.pyfunc
from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

sales = spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "processed_clogs_myn").\
            load()

In [ ]:
sales.select("id").distinct().count()

Out[3]: 44446

In [ ]:
sales = sales.withColumn("old_sales", F.when(F.col("old_sales").isNull(), F.lit(0.0)).otherwise(F.col("old_sales").cast("float")))
sales = sales.withColumn("total_sales", F.when(F.col("total_sales").isNull(), F.lit(0.0)).otherwise(F.col("total_sales").cast("float")))
sales = sales.groupby("id").agg(F.sum("total_sales").alias("total_sales"), F.avg("avg_price").alias("avg_price"), F.avg("max_price").alias("max_price"), F.avg("min_price").alias("min_price"),\
                                F.avg("old_avg_price").alias("old_avg_price"), F.sum("old_sales").alias("old_sales") )

sales.count()

In [ ]:
model_name = "retail_competitive_pricing_model_1"
apply_model_udf = mlflow.pyfunc.spark_udf(spark, f"models:/{model_name}/staging")
    
# Apply the model to the new data
columns = ['old_sales','total_sales','min_price','max_price','avg_price','old_avg_price']
udf_inputs = struct(*columns)
udf_inputs

2023/06/13 07:02:40 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2023/06/13 07:02:40 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
Out[12]: Column<'struct(old_sales, total_sales, min_price, max_price, avg_price, old_avg_price)'>

In [ ]:
sales = sales.fillna(0.0)
sales = sales.withColumn("pred_price",apply_model_udf(udf_inputs))

sales = sales.withColumn("price_elasticity", F.expr("((old_sales - total_sales)/(old_sales + total_sales))/(((old_avg_price - avg_price)+1)/(old_avg_price + avg_price))"))

sales = sales.withColumn("discount", F.ceil((F.lit(1) - F.col("pred_price"))*F.lit(100)))
display(sales)

id,total_sales,avg_price,max_price,min_price,old_avg_price,old_sales,pred_price,price_elasticity,discount
59467,5.0,1.0,1.0,1.0,1.0,4.0,0.9454153832823006,-0.2222222222222222,6
38986,7.0,1.0,1.0,1.0,1.0,6.0,0.9454153832823006,-0.15384615384615385,6
30966,209.0,1.0,1.0,1.0,1.000161144764629,207.0,0.9425635852911128,-0.00961461000576357,6
20868,189.0,1.0,1.013157894736842,0.9868421052631579,1.0077120372904638,188.0,0.9427028481310558,-0.005284740008120823,6
9586,48.0,1.0,1.0,1.0,1.0,46.0,0.9399949980978313,-0.0425531914893617,7
39590,135.0,1.0,1.0,1.0,1.007535083235467,131.0,0.9443862177000628,-0.029962725858953966,6
40740,1.0,1.0,1.0,1.0,0.0,0.0,0.9387022415531037,null,7
30923,7.0,1.0,1.0,1.0,1.0,4.0,0.9454153832823006,-0.5454545454545454,6
46538,5.0,1.0,1.0,1.0,1.0,4.0,0.9454153832823006,-0.2222222222222222,6
15269,2.0,1.0,1.0,1.0,0.0,0.0,0.9388575473933594,null,7


In [ ]:
sales.count()

Out[14]: 44446

In [ ]:
sales.select("id", "pred_price", "price_elasticity").write.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "price_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            mode('overwrite').\
            save()

In [ ]:
sales.orderBy(F.col('discount').desc()).show(10,False)

+-----------+-----------+-------------------+-------------------+-------------------+------------------+---------+--------------------+------------------+------------------+
|product_uid|total_sales|avg_price          |max_price          |min_price          |old_avg_price     |old_sales|pred_price          |price_elasticity  |discount          |
+-----------+-----------+-------------------+-------------------+-------------------+------------------+---------+--------------------+------------------+------------------+
|133161     |0          |50925.9            |50925.9            |50925.9            |0.0               |0        |11386.767400002409  |0.0               |77.64051808607721 |
|129649     |4          |64771.06           |64771.06           |64771.06           |48578.295         |3        |21785.076399999707  |1.0000617597896215|66.36603384289263 |
|124778     |0          |0.16000000000000011|0.16000000000000011|0.16000000000000011|0.1588321167883213|0        |0.06200000003814